# 12: Vehicles per Adult

Model 12 considers the number of autos divided by the number of persons of driving age in the household. (pp. 120)

In [ ]:
import larch

In [ ]:
larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import PX, P, X

for a in [2, 3]:
    m.utility_co[a] = +P("hhinc#2,3") * X("hhinc")
for a in [4, 5, 6]:
    m.utility_co[a] = +P(f"hhinc#{a}") * X("hhinc")

Sometimes we may want to define a part of the utility function that is common across all (or almost all) of the alternatives. We can access a dictionary (more generically called a “mapping”) of alternative codes to alternative names, which can be found via the Dataset.dc.alts_mapping attribute:

In [ ]:
d.dc.alts_mapping

Using this like a standard Python dictionary, we can iterate over all the alternatives, skipping 1, and setting alternative specific constants (ASC’s) for the rest.

In [ ]:
for a, name in d.dc.alts_mapping.items():
    if a == 1:
        continue
    m.utility_co[a] += P("ASC_" + name) + P("vehbyadlt_" + name) * X("numveh/numadlt")

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = (
    +PX("totcost")
    + P("motorized_time") * X("(altid <= 4) * tottime")
    + P("nonmotorized_time") * X("(altid > 4) * tottime")
    + P("motorized_ovtbydist") * X("(altid <= 4) * ovtt/dist")
)

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 12, Autos per Adult"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True, method="bhhh")
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        ".*cost.*",
        ".*time.*",
        ".*dist.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
    ("Ownership", "vehbyadlt.*"),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
from pytest import approx

revealed_x = dict(zip(m.pnames, result.x))
assert result["loglike"] == approx(-3490.3579715315, rel=1e-5), result["loglike"]

expected_x = {
    "ASC_Bike": -1.9627758221667733,
    "ASC_SR2": -1.536865403172976,
    "ASC_SR3+": -3.0235322387181176,
    "ASC_Transit": 1.0496197735578745,
    "ASC_Walk": -0.21728572066754234,
    "hhinc#2,3": -0.0013240180758255255,
    "hhinc#4": -0.004616956633813837,
    "hhinc#5": -0.011663606663415026,
    "hhinc#6": -0.00764683683920611,
    "motorized_ovtbydist": -0.18489944106276082,
    "motorized_time": -0.03803405138642473,
    "nonmotorized_time": -0.046633097590087214,
    "totcost": -0.004179712660535923,
    "vehbyadlt_Bike": -0.6418346308251331,
    "vehbyadlt_SR2": -0.594456791391658,
    "vehbyadlt_SR3+": -0.447408231859455,
    "vehbyadlt_Transit": -1.4092358161539453,
    "vehbyadlt_Walk": -0.7935735138713005,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 0.03), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )